Začneme s Importem knihoven

In [ ]:
import pandas as pd 
import numpy as np
import sklearn.metrics as metrics
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier

# mastavíme jednotný seed pro veškerou náhodnost
seed = 422020

Napřed si načteme a zobrazíme Dataset

In [ ]:
df_data = pd.read_csv("data.csv")
display(df_data)

,ID,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,0,1,3,"Andersson, Mr. August Edvard (""Wennerstrom"")",male,27.0,0,0,350043,7.7958,NaN,S,NaN
1,1,1,3,"Dahl, Mr. Karl Edwart",male,45.0,0,0,7598,8.0500,NaN,S,"Australia Fingal, ND"
2,2,0,2,"Watson, Mr. Ennis Hastings",male,NaN,0,0,239856,0.0000,NaN,S,Belfast
3,3,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,NaN
4,4,0,1,"Meyer, Mr. Edgar Joseph",male,28.0,1,0,PC 17604,82.1708,NaN,C,"New York, NY"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1,1,"Cleaver, Miss. Alice",female,22.0,0,0,113781,151.5500,NaN,S,NaN
996,996,0,2,"Hood, Mr. Ambrose Jr",male,21.0,0,0,S.O.C. 14879,73.5000,NaN,S,"New Forest, England"
997,997,1,1,"Sagesser, Mlle. Emma",female,24.0,0,0,PC 17477,69.3000,B35,C,NaN
998,998,0,3,"Sage, Mr. Frederick",male,NaN,8,2,CA. 2343,69.5500,NaN,S,NaN


In [ ]:
df_evaluation = pd.read_csv("evaluation.csv")
display(df_evaluation)

,ID,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,1000,1,"Wick, Miss. Mary Natalie",female,31.0,0,2,36928,164.8667,C7,S,"Youngstown, OH"
1,1001,1,"Bazzani, Miss. Albina",female,32.0,0,0,11813,76.2917,D15,C,NaN
2,1002,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,"Scituate, MA"
3,1003,3,"Midtsjo, Mr. Karl Albert",male,21.0,0,0,345501,7.7750,NaN,S,NaN
4,1004,3,"O'Connor, Mr. Maurice",male,NaN,0,0,371060,7.7500,NaN,Q,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
304,1304,1,"Lines, Miss. Mary Conover",female,16.0,0,1,PC 17592,39.4000,D28,S,"Paris, France"
305,1305,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S,NaN
306,1306,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,NaN
307,1307,3,"Goodwin, Mr. Charles Edward",male,14.0,5,2,CA 2144,46.9000,NaN,S,"Wiltshire, England Niagara Falls, NY"


Ještě než začnu cokoliv dělat, rozdělím si data na trénovací, validační a testovací.

Rozdělím náš dataset v poměru 60:40 pro TRÉNOVACÍ:VALIDAČNÍ

In [ ]:
data_train, data_valid_tmp = train_test_split(df_data, test_size=0.4, random_state=seed)

Teď rozdělím valdiační data na testovací a validační.

In [ ]:
data_valid, data_test = train_test_split(data_valid_tmp, test_size=0.5, random_state=seed)

Udělám si pole se všemi těmito datasety, abych nemusel psát duplicitní kód.

In [ ]:
combine = [data_train, data_test, data_valid, df_evaluation]

Nyní musíme zjistit, zda naše data jsou "plně vyplněna" a naše příznaky neobsahují NAN hodnoty.

In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 801 to 323
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         600 non-null    int64  
 1   survived   600 non-null    int64  
 2   pclass     600 non-null    int64  
 3   name       600 non-null    object 
 4   sex        600 non-null    object 
 5   age        488 non-null    float64
 6   sibsp      600 non-null    int64  
 7   parch      600 non-null    int64  
 8   ticket     600 non-null    object 
 9   fare       599 non-null    float64
 10  cabin      133 non-null    object 
 11  embarked   600 non-null    object 
 12  home.dest  334 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 65.6+ KB


Převedeme veškeré nečíselné hodnoty na číselné.

# Oprava - stejný medián pro všechny množiny - medián pro "fare" a "embarked

In [ ]:
fare_median = data_train['fare'].mean()
fare_median

32.92242520868114

In [ ]:
embarked_missing = data_train['embarked'].mode().iloc[0]
embarked_missing

'S'

In [ ]:
for data in combine:
    data['sex'] = data['sex'].map( {'female': 1, 'male': 0} ).astype(int)

    data["fare"] = data["fare"].fillna(fare_median)
    data["fare"] = data["fare"].astype("category").cat.codes

    data['embarked'] = data['embarked'].fillna(embarked_missing)
    data['embarked'] = data['embarked'].map( {'Q': 0, 'S': 1, 'C': 2} ).astype(int)

Vidíme, že sloupce "age", "fare", "cabin", "embarked", "home.dest" mají NAN hodnoty. musíme je tedy nejdříve doplnit.

Začnu doplněním sloupce age.

Ze sloupce name si vyberu "oslovení" dané osoby a uložím si jí do nového sloupce title.

Tento sloupec musíme přidat i do validační i testovací množiny pro zachování stejného počtu sloupců.

In [ ]:
for data in combine:
    data['title'] = 'unknown'
    data['title'] = data['name'].str.extract('([A-Za-z]+)\.')
    data['title'].unique()

Zde vidíme všechny možné tituly, které v datasetu máme.

Teď bychom mohli doplnit chybějící věk jako průměr přes všechny tituly.

To ale není úplně optimální řešení. Protože označení Miss může mít jak mladá dívka, tak starší žena.

V následujícím bloku předvedu, jak je rozdílný průměrný věk v závislosti na počtu dětí. (Pokud bude mít 0 rodičů na palubě můžeme očekávat jiný výsledek, než v případě, že má na palubě více rodičů)

In [ ]:
print ("Průměrný věk kohokoliv s titulem Miss:", round(data_train[data_train.title=="Miss"]['age'].mean()))
print ("Průměrný věk kohokoliv s titulem Miss bez rodičů:", round(data_train[(data_train.title=="Miss") & (data_train.parch==0)]['age'].mean()))
print ("Průměrný věk kohokoliv s titulem Miss s rodiči:", round(data_train[(data_train.title=="Miss") & (data_train.parch!=0)]['age'].mean()))

Průměrný věk kohokoliv s titulem Miss: 23
Průměrný věk kohokoliv s titulem Miss bez rodičů: 29
Průměrný věk kohokoliv s titulem Miss s rodiči: 13


Toto tedy provedu pro všechny tituly. Tedy, pro každou osobu, které chybí věk, doplním průměrný věk ze všech, kteří mají stejný titul a 0 / !0 množství rodičů.

In [ ]:
def FillAge(data):
    new_age = []
    for ind in data.index:
        if pd.isna(data['age'][ind]):
            if data['parch'][ind]==0:
                if pd.isna(data[(data.title==data['title'][ind]) & (data.parch==0)]['age'].mean()):
                    new_age.append(round(data['age'].mean()))
                else:
                    new_age.append(round(data[(data.title==data['title'][ind]) & (data.parch==0)]['age'].mean()))
            else:              
                if pd.isna(data[(data.title==data['title'][ind]) & (data.parch!=0)]['age'].mean()):
                    new_age.append(round(data['age'].mean()))
                else:
                    new_age.append(round(data[(data.title==data['title'][ind]) & (data.parch!=0)]['age'].mean()))
        else:
            new_age.append(data['age'][ind])
    return new_age

Ještě máme druhou metodu. Tato metoda dává dohromady lidi, kteřá mají stejné "sex" a jsou ze stejné "pclass". Vezme medián z této skupiny a ten doplní všem lidem ze stejné skupiny lidí, kteří nemají vyplněný věk.

In [ ]:
guess_ages = np.zeros((2,3))
def FillAge_2(data):
    for sex in range(0, 2):
        for pclass in range(0, 3):
            guess_df = data[(data['sex'] == sex) & (data['pclass'] == pclass+1)]['age'].dropna()

            age_guess = guess_df.median()
            guess_ages[sex,pclass] = int( age_guess/0.5 + 0.5 ) * 0.5

# Oprava - stejný medián pro všechny množiny - medián pro "age"

In [ ]:
def FillAgesForAll(data):
    for sex in range(0, 2):
        for pclass in range(0, 3):
            data.loc[(data.age.isnull()) & (data.sex == sex) & (data.pclass == pclass+1), 'age'] = guess_ages[sex,pclass]

    data['age'] = data['age'].astype(int)

In [ ]:
FillAge_2(data_train)
for data in combine:
    FillAgesForAll(data)

In [ ]:
guess_ages

array([[40. , 27. , 25. ],
       [35.5, 28. , 23. ]])

In [ ]:
data_train.head()

,ID,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest,title
801,801,1,1,"Carter, Mr. William Ernest",0,36,1,2,113760,185,B96 B98,1,"Bryn Mawr, PA",Mr
714,714,0,1,"Smart, Mr. John Montgomery",0,56,0,0,113792,111,NaN,1,"New York, NY",Mr
747,747,1,3,"Mamee, Mr. Hanna",0,25,0,0,2677,14,NaN,2,NaN,Mr
316,316,0,1,"Carrau, Mr. Jose Pedro",0,17,0,0,113059,140,NaN,1,"Montevideo, Uruguay",Mr
806,806,0,3,"Henriksson, Miss. Jenny Lovisa",1,28,0,0,347086,25,NaN,1,NaN,Miss


In [ ]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 801 to 323
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         600 non-null    int64 
 1   survived   600 non-null    int64 
 2   pclass     600 non-null    int64 
 3   name       600 non-null    object
 4   sex        600 non-null    int64 
 5   age        600 non-null    int64 
 6   sibsp      600 non-null    int64 
 7   parch      600 non-null    int64 
 8   ticket     600 non-null    object
 9   fare       600 non-null    int16 
 10  cabin      133 non-null    object
 11  embarked   600 non-null    int64 
 12  home.dest  334 non-null    object
 13  title      600 non-null    object
dtypes: int16(1), int64(8), object(5)
memory usage: 66.8+ KB


Nyní máme doplněný sloupec věk.

Teď vyhodím zbytečné sloupce. (Musíme vyhodit stejné sloupce i u validační i testovací množiny, abychom měli stejný počet sloupců)

Sloupec "name" nám žádnou informaci pro predikci nedá. 

Sloupec "home.dest" má mnoho chybějících hodnot. a pokud máme sloupec "fare", nenapadá mne k čemu by byl dobrý. 

Sloupec "ID", protože to je umělý příznak.

Sloupec "ticket" nám asi taky moc neřekne.

In [ ]:
for data in combine:
    data.drop(columns=["name", "home.dest", "ID", "ticket"],inplace=True)

Můžeme si přidal další 2  sloupce, které by nám mohly pomoci s rozhodováním o přežití a to "family_size" a "is_alone"

Tyto 2 sloupce musíme přidat i do validační i testovací množiny pro zachování stejného počtu sloupců. Jelikož jsou pouze validační a testovací, nemusíme je doplňovat skutečnými hodnotami, stačí prázdné.

In [ ]:
for data in combine:
    data['family_size'] = data['sibsp'] + data['parch'] + 1
    # Přičteme jedničku za člověka samotného.
    data['is_alone'] = 0
    data.loc[data['family_size'] == 1, 'is_alone'] = 1

V poslední řadě musíme proměnit sloupec "title" na číselné hodnoty.

Ještě předtím ale zredukuju tituly na menší počet.

In [ ]:
for data in combine:
    data['title'] = data['title'].replace(['Lady', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Rare')
    data['title'] = data['title'].replace('Mlle', 'Miss')
    data['title'] = data['title'].replace('Ms', 'Miss')
    data['title'] = data['title'].replace('Mme', 'Mrs')

In [ ]:
data_train[['title', 'survived']].groupby(['title'], as_index=False).mean()

,title,survived
0,Master,0.448276
1,Miss,0.681818
2,Mr,0.167123
3,Mrs,0.833333
4,Rare,0.166667


In [ ]:
for data in combine:
    data['title'] = data['title'].map({"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5})
    data['title'] = data['title'].fillna(0)

Ještě předspracuji sloupec pro "cabin".

Víme, že první písmeno indikuje palubu. Proto si vezmeme jen první písmeno. Pro chybějící hodnoty doplníme "U" jako nedefinováno. Následně tyto písmena převedema na čísla.

In [ ]:
for data in combine:
    data['cabin'] = data['cabin'].fillna('U')
    data['cabin'] = data['cabin'].astype(str).str[0]

In [ ]:
cabin = {
    'A' : 1,
    'B' : 2,
    'C' : 3,
    'D' : 4,
    'E' : 5,
    'F' : 6,
    'G' : 7,
    'T' : 8,
    'U' : 0
}

for data in combine:
    data['cabin'] = data['cabin'].replace(cabin)

In [ ]:
print(data_train.columns)
print(data_valid.columns)
print(data_test.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin',
       'embarked', 'title', 'family_size', 'is_alone'],
      dtype='object')
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin',
       'embarked', 'title', 'family_size', 'is_alone'],
      dtype='object')
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'cabin',
       'embarked', 'title', 'family_size', 'is_alone'],
      dtype='object')


Konečně máme připravený dataset. Nyní můžeme začít trénovat jednotlivé modely.

# DecisionTree

In [ ]:
x_data = data_train.drop(columns='survived')
y_data = data_train['survived']

x_valid = data_valid.drop(columns='survived')
y_valid = data_valid['survived']

x_test = data_test.drop(columns='survived')
y_test = data_test['survived']

dt = DecisionTreeClassifier()
dt.fit(x_data, y_data)

print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(y_data, dt.predict(x_data))))

accuracy score (train): 0.976667


In [ ]:
print('accuracy score (valid): {0:.6f}'.format(metrics.accuracy_score(y_valid, dt.predict(x_valid))))

accuracy score (valid): 0.675000


Pokud bych neladil žádné hyper parametry,tak mám úšspěšnost pouze 72%, což není dobrý výsledek.

Zkusím tedy poladit Hyper parametry.

In [ ]:
param_grid = {
    'max_depth': range(1,20), 
    'criterion': ['entropy', 'gini'],
    'splitter': ['best', 'random'],
    'max_leaf_nodes': range(2,16)
}
param_comb = ParameterGrid(param_grid)

In [ ]:
val_acc = []
train_acc = []
for params in param_comb:
    dt = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion'], splitter=params['splitter'], max_leaf_nodes=params['max_leaf_nodes'])
    dt.fit(x_data, y_data)
    train_acc.append(metrics.accuracy_score(y_data, dt.predict(x_data)))
    val_acc.append(metrics.accuracy_score(y_valid, dt.predict(x_valid)))

In [ ]:
best_params = param_comb[np.argmax(val_acc)]
print(best_params)
print(val_acc[np.argmax(val_acc)])
print(train_acc[np.argmax(val_acc)])

{'splitter': 'random', 'max_leaf_nodes': 15, 'max_depth': 11, 'criterion': 'entropy'}
0.8
0.8266666666666667


In [ ]:
dt = DecisionTreeClassifier(**best_params)
dt.fit(x_data, y_data)

print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(y_data, dt.predict(x_data))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(y_valid, dt.predict(x_valid))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(y_test, dt.predict(x_test))))

accuracy score (train): 0.821667
accuracy score (validation): 0.760000
accuracy score (test): 0.785000


Skóre nic moc. Vyzkouším další metodu.

# RandomForests

In [ ]:
x_data = data_train.drop(columns='survived')
y_data = data_train['survived']

x_valid = data_valid.drop(columns='survived')
y_valid = data_valid['survived']

x_test = data_test.drop(columns='survived')
y_test = data_test['survived']

rf = RandomForestClassifier()
rf.fit(x_data, y_data)

print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(y_data, rf.predict(x_data))))

accuracy score (train): 0.976667


In [ ]:
print('accuracy score (valid): {0:.6f}'.format(metrics.accuracy_score(y_valid, rf.predict(x_valid))))

accuracy score (valid): 0.750000


In [ ]:
param_grid = {
    'n_estimators': range(1,15), 
    'max_depth': range(1,12),
    'bootstrap': [True, False],
    'min_samples_split': range(2,5),
    'min_samples_leaf': range(2,5)
}
param_comb = ParameterGrid(param_grid)

In [ ]:
val_acc = []
train_acc = []
for params in param_comb:
    rf = RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'], bootstrap=params['bootstrap'], min_samples_split=params['min_samples_split'],min_samples_leaf=params['min_samples_leaf'])
    rf.fit(x_data, y_data)
    train_acc.append(metrics.accuracy_score(y_data, rf.predict(x_data)))
    val_acc.append(metrics.accuracy_score(y_valid, rf.predict(x_valid)))

In [ ]:
best_params = param_comb[np.argmax(val_acc)]
print(best_params)
print(val_acc[np.argmax(val_acc)])
print(train_acc[np.argmax(val_acc)])

{'n_estimators': 5, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_depth': 6, 'bootstrap': True}
0.81
0.8533333333333334


In [ ]:
rf = RandomForestClassifier(**best_params)
rf.fit(x_data, y_data)

print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(y_data, rf.predict(x_data))))
print('accuracy score (validation): {0:.6f}'.format(metrics.accuracy_score(y_valid, rf.predict(x_valid))))
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(y_test, rf.predict(x_test))))

accuracy score (train): 0.858333
accuracy score (validation): 0.770000
accuracy score (test): 0.790000


Druhá metoda doplnění věku se mi osvědčila lépe. Proto jsme jí použil ve výsledném modelu.

## Pro výsledek použiji teda RandomForest a 2. metodu doplnění věku.

In [ ]:
df_evaluation['survived'] = rf.predict(df_evaluation)
df_evaluation['ID'] = range(1000,1309)
df_evaluation[['ID', 'survived']].to_csv('results.csv', index=False)

result = pd.read_csv("results.csv")
result[result['survived'] == 1].count() / result['survived'].count() * 100

ID          34.304207
survived    34.304207
dtype: float64

In [ ]:
result

,ID,survived
0,1000,1
1,1001,1
2,1002,1
3,1003,0
4,1004,0
...,...,...
304,1304,1
305,1305,0
306,1306,0
307,1307,0


In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
from plotly import graph_objects as go


def plot_prediction_results(title, true, predicted):
    fig = go.Figure(data=go.Scatter(x=[0, 100], y=[0, 100], mode="lines", name="y = ŷ"))
    fig.add_scatter(x=true, y=predicted, name="test, prediction", mode="markers")
    fig.update_layout(
        title=f"Visualization of predictions | {title} | MAE: {np.sqrt(mean_absolute_error(predicted, np.array(y_test)))}",
        xaxis_title="Real values",
        yaxis_title="Predicted values",
        
    )

    fig.update_layout(yaxis=dict(range=[0,100],  scaleratio=1));
    fig.update_layout(xaxis=dict(range=[0,100],  scaleratio=1));
    fig.show()


In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_data,y_data)
lr_predicted = lr.predict(x_test)
plot_prediction_results("Linear regression", y_test, lr_predicted)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ec4476fe-a8a1-4f2a-b487-42a7ca65c2c3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>